<table border="0">
    <tr>
        <td>
            <img src="https://ictd2016.files.wordpress.com/2016/04/microsoft-research-logo-copy.jpg" style="width 30px;" />
             </td>
        <td>
            <img src="https://www.microsoft.com/en-us/research/wp-content/uploads/2016/12/MSR-ALICE-HeaderGraphic-1920x720_1-800x550.jpg" style="width 100px;"/></td>
        </tr>
</table>

# Dynamic Double Machine Learning: Use Cases and Examples

Dynamic DoubleML is an extension of the Double ML approach for treatments assigned sequentially over time periods. This estimator will account for treatments that can have causal effects on future outcomes. For more details, see [this paper](https://arxiv.org/abs/2002.07285) or the [EconML docummentation](https://econml.azurewebsites.net/).

For example, the Dynamic DoubleML could be useful in estimating the following causal effects:
* the effect of investments on revenue at companies that receive investments at regular intervals ([see more](https://arxiv.org/abs/2103.08390))
* the effect of prices on demand in stores where prices of goods change over time
* the effect of income on health outcomes in people who receive yearly income

The preferred data format is balanced panel data. Each panel corresponds to one entity (e.g. company, store or person) and the different rows in a panel correspond to different time points. Example:

||Company|Year|Features|Investment|Revenue|
|---|---|---|---|---|---|
|1|A|2018|...|\$1,000|\$10,000|
|2|A|2019|...|\$2,000|\$12,000|
|3|A|2020|...|\$3,000|\$15,000|
|4|B|2018|...|\$0|\$5,000|
|5|B|2019|...|\$100|\$10,000|
|6|B|2020|...|\$1,200|\$7,000|
|7|C|2018|...|\$1,000|\$20,000|
|8|C|2019|...|\$1,500|\$25,000|
|9|C|2020|...|\$500|\$15,000|

(Note: when passing the data to the DynamicDML estimator, the "Company" column above corresponds to the `groups` argument at fit time. The "Year" column above should not be passed in as it will be inferred from the "Company" column)

If group memebers do not appear together, it is assumed that the first instance of a group in the dataset corresponds to the first period of that group, the second instance of the group corresponds to the second period, etc. Example:

||Company|Features|Investment|Revenue|
|---|---|---|---|---|
|1|A|...|\$1,000|\$10,000|
|2|B|...|\$0|\$5,000
|3|C|...|\$1,000|\$20,000|
|4|A|...|\$2,000|\$12,000|
|5|B|...|\$100|\$10,000|
|6|C|...|\$1,500|\$25,000|
|7|A|...|\$3,000|\$15,000|
|8|B|...|\$1,200|\$7,000|
|9|C|...|\$500|\$15,000|

In this dataset, 1<sup>st</sup> row corresponds to the first period of group `A`, 4<sup>th</sup> row corresponds to the second period of group `A`, etc.

In this notebook, we show the performance of the DynamicDML on synthetic and observational data. 

## Notebook Contents

1. [Example Usage with Average Treatment Effects](#1.-Example-Usage-with-Average-Treatment-Effects)
2. [Example Usage with Heterogeneous Treatment Effects](#2.-Example-Usage-with-Heterogeneous-Treatment-Effects)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import econml

In [ ]:
# Main imports
from econml.dynamic.dml import DynamicDML
from econml.tests.dgp import DynamicPanelDGP, add_vlines

# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, MultiTaskLassoCV
import matplotlib.pyplot as plt

%matplotlib inline

# 1. Example Usage with Average Treatment Effects

## 1.1 DGP

We consider a data generating process from a markovian treatment model. 

In the example bellow, $T_t\rightarrow$ treatment(s) at time $t$, $Y_t\rightarrow$outcome at time $t$, $X_t\rightarrow$ features and controls at time $t$ (the coefficients $e, f$ will pick the features and the controls).
\begin{align}
    X_t =& (\pi'X_{t-1} + 1) \cdot A\, T_{t-1} + B X_{t-1} + \epsilon_t\\
    T_t =& \gamma\, T_{t-1} + (1-\gamma) \cdot D X_t + \zeta_t\\
    Y_t =& (\sigma' X_{t} + 1) \cdot e\, T_{t} + f X_t + \eta_t
\end{align}

with $X_0, T_0 = 0$ and $\epsilon_t, \zeta_t, \eta_t \sim N(0, \sigma^2)$. Moreover, $X_t \in R^{n_x}$, $B[:, 0:s_x] \neq 0$ and $B[:, s_x:-1] = 0$, $\gamma\in [0, 1]$, $D[:, 0:s_x] \neq 0$, $D[:, s_x:-1]=0$, $f[0:s_x]\neq 0$, $f[s_x:-1]=0$. We draw a single time series of samples of length $n\_panels \cdot n\_periods$.

In [ ]:
# Define DGP parameters
np.random.seed(123)
n_panels = 5000 # number of panels
n_periods = 2 # number of time periods in each panel
n_treatments = 1 # number of treatments in each period
n_x = 100 # number of features + controls
s_x = 10 # number of controls (endogeneous variables)
s_t = 10 # treatment support size

In [ ]:
# Generate data
dgp = DynamicPanelDGP(n_periods, n_treatments, n_x).create_instance(
            s_x, random_seed=12345, autoreg=1.0)
Y, T, X, W, groups = dgp.observational_data(n_panels, s_t=s_t, random_seed=12345)
true_effect = dgp.true_effect

## 1.2 Train Estimator

In [ ]:
est = DynamicDML(
    model_y=LassoCV(cv=3, max_iter=1000), 
    model_t=MultiTaskLassoCV(cv=3, max_iter=1000), 
    cv=3)

In [ ]:
est.fit(Y, T, X=None, W=W, groups=groups)

In [ ]:
# Average treatment effect of all periods on last period for unit treatments
print(f"Average effect of default policy: {est.ate():0.2f}")

In [ ]:
# Effect of target policy over baseline policy
# Must specify a treatment for each period
baseline_policy = np.zeros((1, n_periods * n_treatments))
target_policy = np.ones((1, n_periods * n_treatments))
eff = est.effect(T0=baseline_policy, T1=target_policy)
print(f"Effect of target policy over baseline policy: {eff[0]:0.2f}")

In [ ]:
# Period treatment effects + interpretation
for i, theta in enumerate(est.intercept_.reshape(-1, n_treatments)):
    print(f"Marginal effect of a treatments in period {i+1} on period {n_periods} outcome: {theta}")

In [ ]:
# Period treatment effects with confidence intervals
est.summary()

In [ ]:
conf_ints = est.intercept__interval(alpha=0.05)

## 1.3 Performance Visualization

In [ ]:
# Some plotting boilerplate code
plt.figure(figsize=(15, 5))
plt.errorbar(np.arange(n_periods*n_treatments)-.04, est.intercept_, yerr=(conf_ints[1] - est.intercept_,
                                                    est.intercept_ - conf_ints[0]), fmt='o', label='DynamicDML')
plt.errorbar(np.arange(n_periods*n_treatments), true_effect.flatten(), fmt='o', alpha=.6, label='Ground truth')
for t in np.arange(1, n_periods):
    plt.axvline(x=t * n_treatments - .5, linestyle='--', alpha=.4)
plt.xticks([t * n_treatments - .5 + n_treatments/2 for t in range(n_periods)],
           ["$\\theta_{}$".format(t) for t in range(n_periods)])
plt.gca().set_xlim([-.5, n_periods*n_treatments - .5])
plt.ylabel("Effect")
plt.legend()
plt.show()

# 2. Example Usage with Heterogeneous Treatment Effects on Time-Invariant Unit Characteristics

We can also estimate treatment effect heterogeneity with respect to the value of some subset of features $X$ in the initial period. Heterogeneity is currently only supported with respect to such initial state features. This for instance can support heterogeneity with respect to time-invariant unit characteristics. In that case you can simply pass as $X$ a repetition of some unit features that stay constant in all periods. You can also pass time-varying features, and their time varying component will be used as a time-varying control. However, heterogeneity will only be estimated with respect to the initial state.

## 2.1 DGP

In [ ]:
# Define additional DGP parameters
het_strength = .5
het_inds = np.arange(n_x - n_treatments, n_x)

In [ ]:
# Generate data
dgp = DynamicPanelDGP(n_periods, n_treatments, n_x).create_instance(
            s_x, hetero_strength=het_strength, hetero_inds=het_inds,
            autoreg=1.0, random_seed=1566)
Y, T, X, W, groups = dgp.observational_data(n_panels, s_t=s_t, random_seed=1)
ate_effect = dgp.true_effect
het_effect = dgp.true_hetero_effect[:, het_inds + 1]

## 2.2 Train Estimator

In [ ]:
est = DynamicDML(
    model_y=LassoCV(cv=3), 
    model_t=MultiTaskLassoCV(cv=3),
    cv=3)

In [ ]:
est.fit(Y, T, X=X, W=W, groups=groups, inference="auto")

In [ ]:
est.summary()

In [ ]:
# Average treatment effect for test points
X_test = X[np.arange(0, 25, 3)]
print(f"Average effect of default policy:{est.ate(X=X_test):0.2f}")

In [ ]:
# Effect of target policy over baseline policy
# Must specify a treatment for each period
baseline_policy = np.zeros((1, n_periods * n_treatments))
target_policy = np.ones((1, n_periods * n_treatments))
eff = est.effect(X=X_test, T0=baseline_policy, T1=target_policy)
print("Effect of target policy over baseline policy for test set:\n", eff)

In [ ]:
# Coefficients: intercept is of shape n_treatments*n_periods
# coef_ is of shape (n_treatments*n_periods, n_hetero_inds).
# first n_treatment rows are from first period, next n_treatment
# from second period, etc.
est.intercept_, est.coef_

In [ ]:
# Confidence intervals
conf_ints_intercept = est.intercept__interval(alpha=0.05)
conf_ints_coef = est.coef__interval(alpha=0.05)

## 2.3 Performance Visualization

In [ ]:
# parse true parameters in array of shape (n_treatments*n_periods, 1 + n_hetero_inds)
# first column is the intercept
true_effect_inds = []
for t in range(n_treatments):
    true_effect_inds += [t * (1 + n_x)] + (list(t * (1 + n_x) + 1 + het_inds) if len(het_inds)>0 else [])
true_effect_params = dgp.true_hetero_effect[:, true_effect_inds]
true_effect_params = true_effect_params.reshape((n_treatments*n_periods, 1 + het_inds.shape[0]))

In [ ]:
# concatenating intercept and coef_
param_hat = np.hstack([est.intercept_.reshape(-1, 1), est.coef_])
lower = np.hstack([conf_ints_intercept[0].reshape(-1, 1), conf_ints_coef[0]])
upper = np.hstack([conf_ints_intercept[1].reshape(-1, 1), conf_ints_coef[1]])

In [ ]:
plt.figure(figsize=(15, 5))
plt.errorbar(np.arange(n_periods * (len(het_inds) + 1) * n_treatments) - .04,
             param_hat.flatten(), yerr=((upper - param_hat).flatten(),
                                        (param_hat - lower).flatten()), fmt='o', label='DynamicDML')
plt.errorbar(np.arange(n_periods * (len(het_inds) + 1) * n_treatments),
             true_effect_params.flatten(), fmt='*', label='Ground Truth')
add_vlines(n_periods, n_treatments, het_inds)
plt.legend()
plt.show()

In [ ]:
est.nuisance_scores_t

In [ ]:
est.nuisance_scores_y

# Job Corps Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV, LogisticRegressionCV


df = pd.read_csv('JC.csv')
df = df.rename(columns={'Unnamed: 0':'id'}).reset_index().drop('index', axis=1).set_index(['id'])
df.head()

In [ ]:
x0_cols = list(df.columns[1:29])
x1_cols = list(df.columns[29:36])
t0_cols = df.columns[[36]]
t1_cols = df.columns[[37]]
y_col = df.columns[43]

In [ ]:
y = df[y_col]
X0 = df[x0_cols]
X1 = df[x1_cols]

In [ ]:
panelX = np.zeros((X0.shape[0], 2, X0.shape[1] + X1.shape[1]))
panelX[:, 0, :X0.shape[1]] = X0.values
panelX[:, 1, :X0.shape[1]] = X0.values
panelX[:, 1, X0.shape[1]:] = X1.values

In [ ]:
panelT = np.zeros((X0.shape[0], 2, 1))
panelT[:, 0, :] = df[t0_cols].values
panelT[:, 1, :] = df[t1_cols].values

In [ ]:
lagpanelT = np.zeros((X0.shape[0], 2, 1))
lagpanelT[:, 1, :] = panelT[:, 0, :]

In [ ]:
panelY = np.zeros((X0.shape[0], 2, 1))
panelY[:, 1, 0] = df[y_col].values

In [ ]:
panelG = np.zeros((X0.shape[0], 2, 1), 'int')
panelG[:, 0, 0] = np.arange(X0.shape[0])
panelG[:, 1, 0] = np.arange(X0.shape[0])

In [ ]:
def long(X):
    return X.reshape((-1, X.shape[-1]))

In [ ]:
from econml.dynamic.dml import DynamicDML
est = DynamicDML(
    model_y=LassoCV(cv=3, max_iter=1000), 
    model_t=LogisticRegressionCV(cv=3, max_iter=2000),
    discrete_treatment=True,
    cv=3)

In [ ]:
est.fit(long(panelY).flatten(), long(panelT).flatten(),
        W=np.hstack([long(panelX), long(lagpanelT)]),
        groups=long(panelG).flatten())

In [ ]:
est.summary()

In [ ]:
est.effect_inference().summary_frame()

# SNMM Algorithm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, MultiTaskLassoCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV, LogisticRegressionCV

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

### Real data

In [ ]:
df = pd.read_csv('JC.csv')
df = df.rename(columns={'Unnamed: 0':'id'}).reset_index().drop('index', axis=1).set_index(['id'])
df.head()

In [ ]:
x0_cols = list(df.columns[1:29])
x1_cols = list(df.columns[29:36])
t0_cols = df.columns[[36]]
t1_cols = df.columns[[37]]
y_col = df.columns[43]

In [ ]:
plt.hist(df['educmum'])
plt.show()

In [ ]:
y = df[y_col].values
X = {0: df[x0_cols], 1: df[x1_cols], 'het': df[x0_cols]}
T = {0: df[t0_cols].values, 1: df[t1_cols].values}
m = 2

In [ ]:
cat = ['age', 'educ', 'educmum', 'educdad']
X[0] = pd.get_dummies(X[0], columns=cat)
X[0] = pd.concat([X[0], df[cat]], axis=1)
x0_cols = list(X[0].columns)

In [ ]:
true_effect_params = np.zeros((m, T[0].shape[1]))

### Simulated Data

In [ ]:
from snmm import gen_data
m = 2
y, X, T, true_effect_params = gen_data(n_periods=m, n_units=10000, n_treatments=2,
                                       n_x=10, s_x=2, s_t=2,
                                       hetero_strenth=.0, n_hetero_vars=0, autoreg=1.0,
                                       instance_seed=13, sample_seed=1)

In [ ]:
X['het'] = X[0]

In [ ]:
true_effect_params

### Simulated data with heterogeneity

In [ ]:
from snmm import gen_data
m = 2
y, X, T, true_effect_params = gen_data(n_periods=m, n_units=10000, n_treatments=1,
                                       n_x=10, s_x=1, s_t=1,
                                       hetero_strenth=.5, n_hetero_vars=1, autoreg=1.0,
                                       instance_seed=123, sample_seed=1)

In [ ]:
true_effect_params

# Analysis

In [ ]:
from snmm import get_linear_model_reg, get_linear_multimodel_reg
from snmm import get_model_reg, get_multimodel_reg
from snmm import get_poly_model_reg, get_poly_multimodel_reg
from econml.utilities import cross_product

### Define Model Parameters

In [ ]:
# model_reg_fn = lambda X, y: get_model_reg(X, y, degrees=[1])
# multimodel_reg_fn = lambda X, y: get_multimodel_reg(X, y, degrees=[1])
# model_reg_fn = get_linear_model_reg
# multimodel_reg_fn = get_linear_multimodel_reg
model_reg_fn = lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=True)
multimodel_reg_fn = lambda X, y: get_poly_multimodel_reg(X, y, degree=1, interaction_only=True)

In [ ]:
het_cols11 = list(X[1].columns)
het_cols10 = list(X[0].columns)
het_cols0 = list(X[0].columns)
def phi(t, X, T, Tt):
    if t == 1:
        return np.hstack([Tt,
                          cross_product(Tt, T[t-1]),
                          cross_product(Tt, X[t][het_cols11].values),
                          cross_product(Tt, X[t-1][het_cols10].values),
                          cross_product(Tt, T[t-1], X[t][het_cols11].values),
                          cross_product(Tt, T[t-1], X[t-1][het_cols10].values)
                         ])
    elif t==0:
        return np.hstack([Tt, cross_product(Tt, X[t][het_cols0].values)])
    raise AttributeError("Not valid")

def phi_names(t):
    if t == 1:
        return ([f't2[{x}]' for x in range(T[1].shape[1])] +
                [f't2[{x}]*t1[{y}]' for y in range(T[0].shape[1]) for x in range(T[1].shape[1])] +
                [f't2[{x}]*x2[{y}]' for y in het_cols11 for x in range(T[1].shape[1])] + 
                [f't2[{x}]*x1[{y}]' for y in het_cols10 for x in range(T[1].shape[1])] +
                [f't2[{x}]*t1[{y}]*{z}' for z in het_cols11 for y in range(T[0].shape[1])
                                        for x in range(T[1].shape[1])] + 
                [f't2[{x}]*t1[{y}]*{z}' for z in het_cols10 for y in range(T[0].shape[1])
                                        for x in range(T[1].shape[1])]
               )
    elif t == 0:
        return ([f't1[{x}]' for x in range(T[1].shape[1])] + 
                [f't1[{x}]*{y}' for y in het_cols0 for x in range(T[1].shape[1])])
    raise AttributeError("Not valid")

def pi(t, X, T):
    return np.ones(T[t].shape)

### Estimate High-Dimensional Linear Blip Model

In [ ]:
from snmm import SNMMDynamicDML

est = SNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                     model_reg_fn=model_reg_fn,
                     model_final_fn=lambda: LassoCV())

In [ ]:
est.fit(X, T, y, pi)

In [ ]:
print(est.policy_value_)

In [ ]:
sig = {}
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        sig[t] = np.abs(est.psi_[t]) > 0.01
        display(est.param_summary(t, coef_thr=0.01).summary_frame())

### Post Selection Inference (not unbiased): Low Dimensional Blip Model

In [ ]:
def phi_sub(t, X, T, Tt):
    return phi(t, X, T, Tt)[:, sig[t]]

def phi_names_sub(t):
    return np.array(phi_names(t))[sig[t]]

In [ ]:
from sklearn.linear_model import LinearRegression

est_sub = SNMMDynamicDML(m=m, phi=phi_sub, phi_names_fn=phi_names_sub,
                         model_reg_fn=lambda X, y: get_model_reg(X, y, degrees=[1]),
                         model_final_fn=lambda: LinearRegression())

In [ ]:
est_sub.fit(X, T, y, pi)

In [ ]:
print(est_sub.policy_value_)

In [ ]:
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        display(est_sub.param_summary(t).summary_frame())

### Policy Delta compared to all zero

For simple phi, where the structural parameters don't change dependent on the target, we can do sth very simple

In [ ]:
print(est_sub.policy_delta_simple_)

### For complex phi we need to re-run the estimation for base

In [ ]:
est_sub.fit_base()

In [ ]:
deltapi, deltapierr = est_sub.policy_delta_complex()

In [ ]:
print(deltapi, deltapierr)

### Optimal Dynamic Policy

In [ ]:
est_sub.fit_opt(X, T, y)

In [ ]:
print(est_sub.opt_policy_delta_simple_)

In [ ]:
print(est_sub.opt_policy_delta_complex())

In [ ]:
for t in range(m):
    print(f'Period {t} effects {true_effect_params[t]}')
    with pd.option_context("precision", 3):
        display(est_sub.opt_param_summary(t).summary_frame())

# Non-Parametric Heterogeneity

In [ ]:
# model_reg_fn = lambda X, y: get_model_reg(X, y, degrees=[1])
# multimodel_reg_fn = lambda X, y: get_multimodel_reg(X, y, degrees=[1])
# model_reg_fn = get_linear_model_reg
# multimodel_reg_fn = get_linear_multimodel_reg
model_reg_fn = lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False)
multimodel_reg_fn = lambda X, y: get_poly_multimodel_reg(X, y, degree=1, interaction_only=True)

In [ ]:
def phi(t, X, T, Tt):
    if t == 1:
        return np.hstack([Tt, cross_product(Tt, T[t-1])])
    elif t==0:
        return np.hstack([Tt])
    raise AttributeError("Not valid")

def phi_names(t):
    if t == 1:
        return ([f't2[{x}]' for x in range(T[1].shape[1])] +
                [f't2[{x}]*t1[{y}]' for y in range(T[0].shape[1]) for x in range(T[1].shape[1])])
    elif t == 0:
        return [f't1[{x}]' for x in range(T[1].shape[1])]
    raise AttributeError("Not valid")

def pi(t, X, T):
    return np.ones(T[t].shape)

In [ ]:
from snmm import fit_heterogeneous_final, LinearModelFinal
from econml.grf import CausalForest
from sklearn.linear_model import LinearRegression
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression

cf_gen = lambda: CausalForest(n_estimators=1000,
                              max_depth=3,
                              min_samples_leaf=50,
                              min_var_fraction_leaf=0.1,
                              min_var_leaf_on_val=True)

In [ ]:
from snmm import HeteroSNMMDynamicDML

het_est = HeteroSNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                               model_reg_fn=lambda X, y: get_model_reg(X, y, degrees=[1]),
                               model_final_fn=cf_gen)

In [ ]:
het_est.fit(X, T, y, pi)

In [ ]:
print(het_est.policy_value_)

In [ ]:
print(het_est.policy_delta_simple_)

In [ ]:
import seaborn as sns
if hasattr(het_est.models_[0], 'feature_importances_'):
    for t in range(m):
        impdf = het_est.feature_importances_(t)
        plt.figure(figsize=(5, 5))
        sns.barplot(y=impdf['name'], x=impdf['importance'])
        plt.show()

In [ ]:
import shap
if hasattr(het_est.models_[0], 'feature_importances_'):
    for t in range(m):
        exp = shap.Explainer(het_est.models_[t])
        shap_values = exp.shap_values(X['het'])
        shap.summary_plot(shap_values, X['het'])

### Fit value of baseline policy, for delta estimation

In [ ]:
het_est.fit_base()

In [ ]:
print(het_est.policy_delta_complex())

### Optimal Dynamic Policy

In [ ]:
het_est.fit_opt(X, T, y)

In [ ]:
het_est.pi_star(1, X, T)[:10]

In [ ]:
if hasattr(het_est.models_[0], 'linear_model'):
    for t in range(m):
        print(f'Period {t} effects {true_effect_params[t]}')
        display(het_est.opt_param_summary(t).summary_frame())

In [ ]:
import seaborn as sns
if hasattr(het_est.opt_models_[0], 'feature_importances_'):
    for t in range(m):
        impdf = het_est.opt_feature_importances_(t)
        plt.figure(figsize=(5, 5))
        sns.barplot(y=impdf['name'], x=impdf['importance'])
        plt.show()

In [ ]:
import shap
if hasattr(het_est.opt_models_[0], 'feature_importances_'):
    for t in range(m):
        exp = shap.Explainer(het_est.opt_models_[t])
        shap_values = exp.shap_values(X['het'])
        shap.summary_plot(shap_values, X['het'])

In [ ]:
print(het_est.opt_policy_value_)

In [ ]:
print(het_est.opt_policy_delta_simple_)

In [ ]:
print(het_est.opt_policy_delta_complex())

### Linear model of heterogeneity

In [ ]:
# linear_gen = lambda: LinearModelFinal(StatsModelsLinearRegression(fit_intercept=False),
#                                       lambda x: x)
linear_gen = lambda: LinearModelFinal(LassoCV(fit_intercept=False),
                                  lambda x: x)
het_est.model_final_fn = linear_gen

In [ ]:
het_est.fit_final()

In [ ]:
if hasattr(het_est.models_[0], 'linear_model'):
    for t in range(m):
        print(f'Period {t} effects {true_effect_params[t]}')
        display(het_est.param_summary(t).summary_frame())

# Many experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from experiment import main

res = main(n_periods=2, n_units=10000, n_treatments=1, n_x=2, s_x=1, s_t=1,
           n_instances=1, n_samples=1)

In [ ]:
res

In [ ]:
from experiment import main
import joblib
n_instances = 2
n_samples = 8
n_periods = 2
for n_hetero_vars in [0]:
    for n_units in [1000]:
        for n_x in [2]:
            res = main(n_periods=n_periods, n_units=n_units, n_treatments=1,
                       n_x=n_x, s_x=2, s_t=2,
                       n_hetero_vars=n_hetero_vars,
                       n_instances=n_instances,
                       n_samples=n_samples,
                       verbose=1)
            file = f'n_ins_{n_instances}_n_sam_{n_samples}_n_hetero_vars_{n_hetero_vars}_n_units_{n_units}_n_x_{n_x}.jbl'
            joblib.dump(res, file)

In [ ]:
from experiment import all_experiments
all_experiments()

# Post-processing many experiments

In [ ]:
import joblib

def helper(df, prop):
    return df[prop] if prop in df else 0

n_instances = 2
n_samples = 8
n_hetero_vars = 0
n_periods = 2
n_units = 1000
n_x = 2
file = f'n_ins_{n_instances}_n_sam_{n_samples}_n_hetero_vars_{n_hetero_vars}_n_units_{n_units}_n_x_{n_x}.jbl'
res = joblib.load(file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def metrics(truth, point, std):
    metrics = {}
    metrics['coverage90'] = np.mean((point - 1.645 * std <= truth) & (point + 1.645 * std >= truth))
    metrics['coverage95'] = np.mean((point - 1.96 * std <= truth) & (point + 1.96 * std >= truth))
    metrics['coverage99'] = np.mean((point - 2.58 * std <= truth) & (point + 2.58 * std >= truth))
    metrics['bias'] = np.mean(point - truth)
    metrics['rmse'] = np.sqrt(np.mean((point - truth)**2))
    metrics['mae'] = np.mean(np.abs(point - truth))
    metrics['mape'] = np.mean(np.abs(point - truth) / np.abs(truth))
    return metrics

summary = {}
for instance in range(n_instances):
    summary[instance] = {}
    for feat in res[instance][0]['true']:
        if feat == 'params':
            continue
        summary[instance][feat] = {}
        truth = res[instance][0]['true'][feat]
        summary[instance][feat]['true'] = truth
        for method in res[instance][0]:
            if method == 'true':
                continue
            if feat in res[instance][0][method]:
                summary[instance][feat][method] = {}
                point = np.array([res[instance][t][method][feat][0] for t in range(n_samples)])
                std = np.array([res[instance][t][method][feat][1] for t in range(n_samples)])
                summary[instance][feat][method] = metrics(truth, point, std)
    
    summary[instance]['params'] = {}
    for period in range(n_periods):
        summary[instance]['params'][period] = {}
        for feat_it, feat in enumerate(res[instance][0]['true']['params'][period].keys()):
            if res[instance][0]['true']['params'][period][feat] == 0:
                continue
            summary[instance]['params'][period][feat] = {}
            truth = res[instance][0]['true']['params'][period][feat]
            summary[instance]['params'][period][feat]['true'] = truth
            for method in res[instance][0]:
                if method == 'true':
                    continue
                point = np.array([helper(res[instance][t][method]['params'][period]['point_estimate'], feat)
                                  for t in range(n_samples)])
                std = np.array([helper(res[instance][t][method]['params'][period]['stderr'], feat)
                                for t in range(n_samples)])
                summary[instance]['params'][period][feat][method] = metrics(truth, point, std)
                plt.hist(point, label=method)
            plt.axvline(x = truth, label='true', color='red')
            plt.title(f'period {period}, feat {feat}, instance {instance}')
            plt.legend()
            plt.show()

In [ ]:
allsummary = {}
for feat in summary[0]:
    if feat == 'params':
        continue
    allsummary[feat] = {}
    for method in summary[0][feat]:
        if method =='true':
            continue
        allsummary[feat][method] = {}
        for attr in summary[0][feat][method]:
            avg = np.mean([summary[t][feat][method][attr]
                           for t in range(n_instances)])
            std = np.std([summary[t][feat][method][attr]
                          for t in range(n_instances)])
            stderr = std / np.sqrt(n_instances)
            allsummary[feat][method][attr] = "{:.3f} +/- {:.3f}".format(avg, 1.96 * stderr)

allsummary['params'] = {}
for period in summary[0]['params']:
    allsummary['params'][period] = {}
    for feat in summary[0]['params'][period]:
        allsummary['params'][period][feat] = {}
        for method in summary[0]['params'][period][feat]:
            if method =='true':
                continue
            allsummary['params'][period][feat][method] = {}
            for attr in summary[0]['params'][period][feat][method]:
                avg = np.mean([summary[t]['params'][period][feat][method][attr]
                               for t in range(n_instances)])
                std = np.std([summary[t]['params'][period][feat][method][attr]
                              for t in range(n_instances)])
                stderr = std / np.sqrt(n_instances)
                allsummary['params'][period][feat][method][attr] = "{:.3f} +/- {:.3f}".format(avg, 1.96 * stderr)

In [ ]:
allsummary